In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### ANN for MNIST dataset

In [ ]:
#importing libraires
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets,transforms
from torch import nn
from torch.nn import Linear 

In [ ]:
transform = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
training_datasets = datasets.MNIST(root='',download=True,train=True,transform=transform)
test_datasets = datasets.MNIST(root='',download=True,train=False,transform=transform)
training_dataloader = torch.utils.data.DataLoader(dataset=training_datasets,shuffle=True,batch_size=100)
test_dataloader = torch.utils.data.DataLoader(dataset=test_datasets,shuffle=True,batch_size=100)

In [ ]:
def im_convert(tensor):
    image = tensor.numpy()
    image = image.transpose(1,2,0)
    image = image *np.array(0.5) + np.array(0.5)
    image = image.clip(0,1)
    return image[:,:,0]

In [ ]:
data_iter = iter(training_dataloader)
images,labels = data_iter.next()

fig = plt.figure(figsize=(25,4))

for idx in np.arange(10):
    ax = fig.add_subplot(2,10,idx+1)
    plt.imshow(im_convert(images[idx]))
    ax.set_title([labels[idx].item()])

In [ ]:
# Defining ANN
class Classifier(nn.Module):
    def __init__(self,input_size,H1,H2,H3,output_size):
        super().__init__()
        self.linear1 = Linear(input_size,H1)
        self.linear2 = Linear(H1,H2)
        self.linear3 = Linear(H2,H3)
        self.linear4 = Linear(H3,output_size)
        
    def forward(self,x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        x = self.linear4(x)
        return x
    def predict(self,x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        x = self.linear4(x)
        return x

In [ ]:
model = Classifier(784,512,256,64,10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [ ]:
epochs = 12
train_losses = []
train_accuracy = []

for epoch in range(epochs):
    batch_loss = []
    batch_accuracies = []
    for inputs,labels in training_dataloader:
        inputs = inputs.view(inputs.shape[0],-1)
        Y_pred = model.forward(inputs)
        loss = criterion(Y_pred,labels)
        batch_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #batch accuracy
        _,pred = torch.max(Y_pred,1)
        batch_accuracy = torch.sum(pred == labels.data).item()
        batch_accuracies.append(batch_accuracy)
    
    test_batch_loss = []
    test_batch_accuracies = []
    for test_inputs,test_labels in test_dataloader:
        test_inputs = test_inputs.view(test_inputs.shape[0],-1)
        predictions = model.predict(test_inputs)
        test_loss = criterion(predictions,test_labels)
        test_batch_loss.append(test_loss.item())
        _,test_pred = torch.max(predictions,1)
        test_batch_accuracy = torch.sum(pred == test_labels.data).item()
        test_batch_accuracies.append(batch_accuracy)
        
    #for training        
    mean_epochloss = np.array(batch_loss).mean()
    mean_epoch_accuracy = np.array(batch_accuracies).mean()
    #for testing
    test_mean_epochloss = np.array(test_batch_loss).mean()
    test_mean_epoch_accuracy = np.array(test_batch_accuracies).mean()
    print('for epoch {} ,training loss {:.2f}/accuracy {:.2f} , Test loss {:.2f}/accuracy as {:.2f}'.format(epoch,mean_epochloss,mean_epoch_accuracy,test_mean_epochloss,test_mean_epoch_accuracy))
    train_losses.append(mean_epochloss)
    train_accuracy.append(mean_epoch_accuracy)
    
    #training accuracy
    
    


In [ ]:
#loss
plt.plot(range(epochs),train_losses)
#plt.plot(range(epochs),test_mean_epochloss)
plt.legend()
plt.show()

In [ ]:
#predicting images
import requests
from PIL import Image
from PIL import ImageOps
url = 'https://images.homedepot-static.com/productImages/007164ea-d47e-4f66-8d8c-fd9f621984a2/svn/architectural-mailboxes-house-letters-numbers-3585b-5-64_1000.jpg'
response = requests.get(url,stream = True)
img = Image.open(response.raw)
img = ImageOps.invert(img)
img = img.convert('1')
img = transform(img)
plt.imshow(im_convert(img))


In [ ]:
test_input = img.view(img.shape[0],-1)
prediction = model.predict(test_input)
_,test_pred = torch.max(prediction,1)
print(test_pred.item())